In [1]:
!pip install fsspec s3fs japanize_matplotlib boto3 > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 neologdn > /dev/null

!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [3]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
from datetime import datetime
import numpy as np
from fastprogress import progress_bar as pb
import seaborn as sns
import MeCab
import neologdn
import unicodedata
sns.set(font="IPAexGothic")

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
import boto3
from boto3.dynamodb.conditions import Key
from typing import List, Dict

In [6]:
m_wakati = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
m_chasen = MeCab.Tagger('-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

In [84]:
df_btc_return = pd.read_csv('/content/drive/MyDrive/data/crypto/historical/gmo/return_btc_5min.csv')
df_btc_return['timestamp'] = pd.to_datetime(df_btc_return['timestamp'])
df_btc_return.set_index('timestamp', inplace=True)

In [85]:
df_btc_return

,size_buy,price_buy,size_sell,price_sell,return_1h,return_3h,return_6h,return_12h,return_18h,return_24h,return_48h,return_72h,return_120h,return_168h,return_720h,return_1440h,return_2160h
timestamp,,,,,,,,,,,,,,,,,
2019-12-31 21:15:00,0.003475,777540.00,0.012000,7.775560e+05,1.000916,1.000649,1.006713,1.002986,1.007373,1.008283,0.968452,1.017735,1.029784,1.132307,1.336909,1.201652,0.905107
2019-12-31 21:20:00,0.001000,777580.00,0.018333,7.776577e+05,1.000900,1.000561,1.007009,1.003472,1.006925,1.008224,0.968900,1.016635,1.029214,1.130117,1.336218,1.199488,0.904578
2019-12-31 21:25:00,0.001367,777862.00,0.024667,7.777593e+05,1.000971,1.000161,1.007203,1.002903,1.006875,1.007851,0.969047,1.015219,1.027670,1.129068,1.335293,1.198884,0.905345
2019-12-31 21:30:00,0.001000,777862.00,0.031000,7.778610e+05,1.001404,1.000124,1.007762,1.003104,1.007004,1.007844,0.969545,1.014171,1.026500,1.126282,1.334464,1.199147,0.903535
2019-12-31 21:35:00,0.001000,777830.00,0.065500,7.776455e+05,1.001878,1.000129,1.007502,1.003347,1.007482,1.007878,0.970083,1.013165,1.025095,1.125580,1.332712,1.199459,0.903243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01 20:30:00,0.000158,5463890.25,0.004438,5.463901e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 20:35:00,0.000100,5469121.00,0.005000,5.469120e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 20:40:00,0.003750,5465023.00,0.010000,5.462800e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
df_btc_return

,size_buy,price_buy,size_sell,price_sell,return_1h,return_3h,return_6h,return_12h,return_18h,return_24h,return_48h,return_72h,return_120h,return_168h,return_720h,return_1440h,return_2160h
timestamp,,,,,,,,,,,,,,,,,
2019-12-31 21:15:00,0.003475,777540.00,0.012000,7.775560e+05,0.000916,0.000649,0.006713,0.002986,0.007373,0.008283,-0.031548,0.017735,0.029784,0.132307,0.336909,0.201652,-0.094893
2019-12-31 21:20:00,0.001000,777580.00,0.018333,7.776577e+05,0.000900,0.000561,0.007009,0.003472,0.006925,0.008224,-0.031100,0.016635,0.029214,0.130117,0.336218,0.199488,-0.095422
2019-12-31 21:25:00,0.001367,777862.00,0.024667,7.777593e+05,0.000971,0.000161,0.007203,0.002903,0.006875,0.007851,-0.030953,0.015219,0.027670,0.129068,0.335293,0.198884,-0.094655
2019-12-31 21:30:00,0.001000,777862.00,0.031000,7.778610e+05,0.001404,0.000124,0.007762,0.003104,0.007004,0.007844,-0.030455,0.014171,0.026500,0.126282,0.334464,0.199147,-0.096465
2019-12-31 21:35:00,0.001000,777830.00,0.065500,7.776455e+05,0.001878,0.000129,0.007502,0.003347,0.007482,0.007878,-0.029917,0.013165,0.025095,0.125580,0.332712,0.199459,-0.096757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01 20:30:00,0.000158,5463890.25,0.004438,5.463901e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 20:35:00,0.000100,5469121.00,0.005000,5.469120e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 20:40:00,0.003750,5465023.00,0.010000,5.462800e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
df_news = pd.read_csv('/content/drive/MyDrive/data/google_news_20200301_20200822.csv', parse_dates=['published_date'])

In [106]:
df_news['published_date'] = pd.to_datetime(df_news['published_date'])
df_news['published_jst'] = pd.to_datetime(df_news['published'].map(lambda x: x.replace('+09:00', '')))

In [107]:
df_news

,published,published_gmt,published_date,title,summary,topic,published_jst
0,2020-03-02 08:52:05+09:00,2020-03-01 23:52:05+00:00,2020-03-02,100倍ズームカメラスマホやパカパカスマホを触り放題。掘り出し物を探しに西海岸のサムスン体験...,100倍ズームカメラスマホやパカパカスマホを触り放題。掘り出し物を探しに西海岸のサムスン体験...,TECHNOLOGY,2020-03-02 08:52:05
1,2020-03-02 08:52:00+09:00,2020-03-01 23:52:00+00:00,2020-03-02,10～12月の全産業設備投資は3.5％減、13期ぶりマイナス - ブルームバーグ,10～12月の全産業設備投資は3.5％減、13期ぶりマイナス ブルームバーグ設備投資、約3...,BUSSINESS,2020-03-02 08:52:00
2,2020-03-02 08:52:00+09:00,2020-03-01 23:52:00+00:00,2020-03-02,10～12月全産業設備投資､13期ぶりマイナス-世界経済減速や台風で - ブルームバーグ,10～12月全産業設備投資､13期ぶりマイナス-世界経済減速や台風で ブルームバーグ日本企...,BUSSINESS,2020-03-02 08:52:00
3,2020-03-02 06:51:00+09:00,2020-03-01 21:51:00+00:00,2020-03-02,19歳FWビニシウスが“メッシ超え”決勝弾!! R・マドリー、天王山クラシコ制して首位奪還 ...,19歳FWビニシウスが“メッシ超え”決勝弾!! R・マドリー、天王山クラシコ制して首位奪還 ...,SPORTS,2020-03-02 06:51:00
4,2020-03-01 11:00:00+09:00,2020-03-01 02:00:00+00:00,2020-03-01,1位のタピオカは「ほぼ専門店」の味!? カルディで見つけた「タピオカミルクティー」飲み比べ ...,1位のタピオカは「ほぼ専門店」の味!? カルディで見つけた「タピオカミルクティー」飲み比べ ...,BUSSINESS,2020-03-01 11:00:00
...,...,...,...,...,...,...,...
42119,2020-08-22 10:33:22+09:00,2020-08-22 01:33:22+00:00,2020-08-22,コロナ終息は「2年以内」 WHOテドロス氏が言及 [新型コロナウイルス] - 朝日新聞デジタル,コロナ終息は「2年以内」 WHOテドロス氏が言及 [新型コロナウイルス] 朝日新聞デジタル...,WORLD,2020-08-22 10:33:22
42120,2020-08-22 10:33:22+09:00,2020-08-22 01:33:22+00:00,2020-08-22,コロナ終息は「2年未満で」 WHOテドロス氏が言及 [新型コロナウイルス] - 朝日新聞デジタル,コロナ終息は「2年未満で」 WHOテドロス氏が言及 [新型コロナウイルス] 朝日新聞デジタ...,WORLD,2020-08-22 10:33:22
42121,2020-08-22 12:10:00+09:00,2020-08-22 03:10:00+00:00,2020-08-22,サツキは児童虐待？「となりのトトロ」2年ぶりの地上波放送で、疑問の声噴出 - リアルライブ,サツキは児童虐待？「となりのトトロ」2年ぶりの地上波放送で、疑問の声噴出 リアルライブ,ENTERTAINMENT,2020-08-22 12:10:00
42122,2020-08-22 12:10:00+09:00,2020-08-22 03:10:00+00:00,2020-08-22,サツキは児童虐待？「となりのトトロ」放送で時流を反映した指摘 - livedoor,サツキは児童虐待？「となりのトトロ」放送で時流を反映した指摘 livedoor「となりのト...,ENTERTAINMENT,2020-08-22 12:10:00


In [108]:
df_news['published_jst_5min'] = df_news['published_jst'].dt.floor('5T')

In [109]:
def preprocess_sentence(sentence):
    sentence = neologdn.normalize(sentence)
    sentence = sentence.lower()
    sentence = unicodedata.normalize('NFKC', sentence)
    return sentence

In [65]:
word_list = []
skipped_date_set = set()
target_datetimes = []
datetime_list = []
exclude_hinshi_list = ['記号', 'サ変接続', '助詞']

for pub_datetime in pb(df_news['published_jst_5min'].unique()):
    word_set = set()
    df_news_day = df_news[df_news['published_jst_5min'] == pub_datetime]
    try:
        for summary in df_news_day['summary']:
            sentence = preprocess_sentence(summary)
            df_word = pd.DataFrame(
                [item.split('\t') for item in m_chasen.parse(sentence).split('\n') if item not in ('EOS', '')],
                columns=['表層形', '発音', '原型', '形態素の品詞型', '活用形', '活用型']
            )
            df_word = df_word[~df_word['形態素の品詞型'].map(lambda x: any( [hinshi in x for hinshi in exclude_hinshi_list] ))]
            df_word = df_word[df_word['表層形'].str.contains(r"[一-龯ぁ-んァ-ンA-Za-z々ゝゞヽヾヴヵヶ]")]
            df_word = df_word[~df_word['表層形'].str.contains(r"[0-9]")]
            word_set.update(df_word['表層形'].tolist())
        # for lag in rtn_lags:
        #     lag_rtns[f'rtn_lag{lag}'] += [df_stock_for_news.loc[pd.to_datetime(pub_datetime), f'rtn_lag{lag}']] * len(word_set)
        datetime_list += [pub_datetime] * len(word_set)
        word_list += list(word_set)
        target_datetimes.append(pub_datetime)
    except Exception as e:
        print(e)
        skipped_date_set.add(pub_datetime)
        continue

In [66]:
df_word = pd.DataFrame()
df_word['word'] = word_list
df_word['datetime'] = datetime_list

In [96]:
df_word

,word,datetime
0,西海岸,2020-03-02 08:50:00
1,探し,2020-03-02 08:50:00
2,パカパカスマホ,2020-03-02 08:50:00
3,台風,2020-03-02 08:50:00
4,掘り出し物,2020-03-02 08:50:00
...,...,...
999960,コロナ感染,2020-08-22 22:40:00
999961,諦め,2020-08-22 22:40:00
999962,見る,2020-08-22 22:40:00
999963,.jp,2020-08-22 22:40:00


In [110]:
def create_word_return_df(
    df_word,
    df_btc_return,
):
    df_word_return = pd.merge(
        df_word, df_btc_return,
        how='left',
        left_on='datetime', right_index=True
    )
    return df_word_return.dropna()

In [111]:
df_word_return = create_word_return_df(df_word, df_btc_return)

In [112]:
df_word_return

,word,datetime,size_buy,price_buy,size_sell,price_sell,return_1h,return_3h,return_6h,return_12h,return_18h,return_24h,return_48h,return_72h,return_120h,return_168h,return_720h,return_1440h,return_2160h
0,西海岸,2020-03-02 08:50:00,0.00965,9.358611e+05,0.013557,9.360349e+05,0.001318,0.006079,0.016411,0.029423,0.024041,0.015295,0.013133,0.02301,0.027845,-0.126848,-0.272640,0.005865,0.098565
1,探し,2020-03-02 08:50:00,0.00965,9.358611e+05,0.013557,9.360349e+05,0.001318,0.006079,0.016411,0.029423,0.024041,0.015295,0.013133,0.02301,0.027845,-0.126848,-0.272640,0.005865,0.098565
2,パカパカスマホ,2020-03-02 08:50:00,0.00965,9.358611e+05,0.013557,9.360349e+05,0.001318,0.006079,0.016411,0.029423,0.024041,0.015295,0.013133,0.02301,0.027845,-0.126848,-0.272640,0.005865,0.098565
3,台風,2020-03-02 08:50:00,0.00965,9.358611e+05,0.013557,9.360349e+05,0.001318,0.006079,0.016411,0.029423,0.024041,0.015295,0.013133,0.02301,0.027845,-0.126848,-0.272640,0.005865,0.098565
4,掘り出し物,2020-03-02 08:50:00,0.00965,9.358611e+05,0.013557,9.360349e+05,0.001318,0.006079,0.016411,0.029423,0.024041,0.015295,0.013133,0.02301,0.027845,-0.126848,-0.272640,0.005865,0.098565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999960,コロナ感染,2020-08-22 22:40:00,0.04920,1.235480e+06,0.077467,1.234866e+06,0.001348,-0.001484,-0.004190,-0.007801,-0.003209,-0.000112,0.008191,-0.01956,-0.026116,-0.017224,-0.107938,0.106727,0.563611
999961,諦め,2020-08-22 22:40:00,0.04920,1.235480e+06,0.077467,1.234866e+06,0.001348,-0.001484,-0.004190,-0.007801,-0.003209,-0.000112,0.008191,-0.01956,-0.026116,-0.017224,-0.107938,0.106727,0.563611
999962,見る,2020-08-22 22:40:00,0.04920,1.235480e+06,0.077467,1.234866e+06,0.001348,-0.001484,-0.004190,-0.007801,-0.003209,-0.000112,0.008191,-0.01956,-0.026116,-0.017224,-0.107938,0.106727,0.563611
999963,.jp,2020-08-22 22:40:00,0.04920,1.235480e+06,0.077467,1.234866e+06,0.001348,-0.001484,-0.004190,-0.007801,-0.003209,-0.000112,0.008191,-0.01956,-0.026116,-0.017224,-0.107938,0.106727,0.563611


In [113]:
df_word_return[df_word_return['return_1h'].isnull()].datetime.unique()

array([], dtype='datetime64[ns]')

In [133]:
def create_word_score_df(df_word_return):
    return df_word_return.groupby('word').agg(['sum', 'mean', 'count']).sort_values(by=('return_12h', 'mean'))

In [134]:
df_word_score = create_word_score_df(df_word_return)

In [135]:
MIN_WORD_OCCURANCE_COUNT = 10

In [136]:
pd.set_option('display.max_rows', 100)
filter_condition = df_word_score[('return_12h', 'count')] >= MIN_WORD_OCCURANCE_COUNT
df_word_score[filter_condition][[cols for cols in df_word_score.columns if 'return' in cols[0]]].sort_values(by=('return_12h', 'mean'))

return_1h                  ... return_2160h                
                   sum      mean count  ...          sum      mean count
word                                    ...                             
annnewschwho -0.127656 -0.006079    21  ...     6.742346  0.321064    21
弱気           -0.217961 -0.018163    12  ...     3.863438  0.321953    12
深澤辰哉         -0.041318 -0.004132    10  ...     5.192353  0.519235    10
藤岡康           0.035382  0.003538    10  ...     2.479661  0.247966    10
負かす          -0.011600 -0.001160    10  ...     2.160689  0.216069    10
...                ...       ...   ...  ...          ...       ...   ...
割り込む          0.087408  0.007284    12  ...     5.398652  0.449888    12
ディオール         0.040923  0.004092    10  ...     5.178538  0.517854    10
量的緩和         -0.032016 -0.002668    12  ...     8.226566  0.685547    12
システム設計        0.065364  0.005028    13  ...     9.342349  0.718642    13
取引停止         -0.072830 -0.006621    11  ...     7.889025  0.717184    11

[11499 rows x 39 columns]

In [137]:
# 12時間後リターンが下降傾向単語トップ100
df_word_score[filter_condition][[cols for cols in df_word_score.columns if 'return' in cols[0]]].sort_values(by=('return_12h', 'mean')).index[:100]

Index(['annnewschwho', '弱気', '深澤辰哉', '藤岡康', '負かす', '緩める', '国際社会', 'ジャズ', '樹',
       '調整中', '立川志らく', 'お天気キャスター', 'オリジナルアニメ', '緊急会見', '柄本', '気象予報士', 'ドゥカティ',
       '競走馬', 'ピクセル', 'パスコード', 'クロップ監督', 'フェア', '遠い', 'アドマイヤビルゴ', 'バド', '味わえる',
       '金髪', '公式ページ', '一足', 'コマンド', '田中樹', 'ビザ', '酒場', 'ボンド', '大幅反発', '竈門',
       'ビートたけし', 'discord', '独身', 'power', '棒', '京都新聞杯', '人馬', 'jfe', '右足',
       '長野県', 'コミケ', 'ヨガ', '支持者', 'ハーランド', 'ゲノセクト', 'ソロ曲', '宣戦布告', 'パブリッシング',
       '炉', '結婚式', 'ぐるナイ', 'つかん', '争点', '正社員', 'ホース', 'preview', '疫病', '無罪',
       '本能寺の変', 'フォーク', 'west', 'ポケ', '申し訳', '吉高由里子', 'スカート', '飛び込ん', '対戦相手',
       '世界最大', 'よーく', '金鯱賞', 'ベル', '財産', '情報誌', 'ほしのすな', '愚か', '子役', '小宮璃央',
       'remedy', '祭典', 'ktm', '世界情勢', 'ギラティナ', 'ノーカット', '順当', 'センバツ', '保',
       '再発防止', '猫ちゃん', 'やめよ', 'ハイペース', '聖火', '中韓', '打て', '初対面'],
      dtype='object', name='word')

In [141]:
# 12時間後リターンが上昇傾向単語トップ100
df_word_score[filter_condition][[cols for cols in df_word_score.columns if 'return' in cols[0]]].sort_values(by=('return_12h', 'mean')).index[-100:][::-1]

Index(['取引停止', 'システム設計', '量的緩和', 'ディオール', '割り込む', 'ハードウェア', '裏技', 'itmediaau',
       '見つける', '超高速', '乗車率', '解説動画', '甦る', 'ジョーカナチャン', 'ウォッチドッグス',
       'フラッグシップスマホ', 'pcエンジン', 'xperia', 'ジェル', '偲ぶ', '名古屋駅', '小ネタ', 'カスタム',
       'ジェット', 'チャンピオンズリーグ', 'ランドロス', '通常版', 'お酒', '初め', '電話会議', 'ライザ', '持ち込ん',
       '中銀', 'ロード', '根強く', 'いち', '家政夫のミタゾノ', '見放題', 'b/s', 'iphoneアプリ',
       'ダウ平均株価', 'ファクトリー', 'アゼルバイジャン', 'モナコgp', 'find', 'へん', 'くださっ', '会員登録',
       '選び方', '試', '重し', 'ニューシングル', 'windows update', 'エイプリルフール', 'シカゴ', '後任',
       '考え方', '強制力', '爆速', 'aquos', 'blade', '次週', 'warframe', 'battle', '慎',
       '伝説ポケモン', 'パドック', 'ワイド', '立ち位置', '威力', 'クッキー', '時間制限', '売る', 'ひるおび',
       '理事会', '教室', '勢力', '下落率', '吉本興業', '意見交換', 'ジェム', 'トラックパッド', 'クアッドカメラ',
       'lightning', 'fitbit', '小澤征悦', '無制限', '投資家', '消息', '切り開く', 'uuum',
       '仲良く', '本家', 'icu', '吉本', '香り', 'アトリエ', 'たぬきち', '画', 'ソーシャル'],
      dtype='object', name='word')

In [144]:
df_news[df_news.summary.str.contains('取引停止')]

,published,published_gmt,published_date,title,summary,topic,published_jst,published_jst_5min
1389,2020-03-10 05:39:54+09:00,2020-03-09 20:39:54+00:00,2020-03-10,NYダウ、史上最大の下げ幅 取引の一時停止が初発動 - 朝日新聞,NYダウ、史上最大の下げ幅 取引の一時停止が初発動 朝日新聞NYダウ一時2000ドル超安 ...,BUSSINESS,2020-03-10 05:39:54,2020-03-10 05:35:00
1401,2020-03-10 02:00:00+09:00,2020-03-09 17:00:00+00:00,2020-03-10,NY株、一時2000ドル超安 取引停止措置を発動 - 日本経済新聞,NY株、一時2000ドル超安 取引停止措置を発動 日本経済新聞ＮＹ株急落で開始、一時２００...,BUSSINESS,2020-03-10 02:00:00,2020-03-10 02:00:00
1402,2020-03-10 00:32:51+09:00,2020-03-09 15:32:51+00:00,2020-03-10,NY株が急落、取引一時停止 - Yahoo!ニュース,NY株が急落、取引一時停止 Yahoo!ニュースNYダウ急落 取引が自動停止に(20/03...,BUSSINESS,2020-03-10 00:32:51,2020-03-10 00:30:00
2101,2020-03-12 22:52:00+09:00,2020-03-12 13:52:00+00:00,2020-03-12,NYダウ急落、一時2200ドル超安 売買一時停止 - 朝日新聞,NYダウ急落、一時2200ドル超安 売買一時停止 朝日新聞NY株急落 一時2100ドル以上...,BUSSINESS,2020-03-12 22:52:00,2020-03-12 22:50:00
2105,2020-03-13 06:26:24+09:00,2020-03-12 21:26:24+00:00,2020-03-13,NY株式市場再び取引停止 終値で史上最大また更新(20/03/13) - ANNnewsCH,NY株式市場再び取引停止 終値で史上最大また更新(20/03/13) ANNnewsCHN...,BUSSINESS,2020-03-13 06:26:24,2020-03-13 06:25:00
2106,2020-03-13 00:41:09+09:00,2020-03-12 15:41:09+00:00,2020-03-13,NY株急落 一時2100ドル以上の下落(20/03/13) - ANNnewsCH,NY株急落 一時2100ドル以上の下落(20/03/13) ANNnewsCHダウ平均株価...,BUSSINESS,2020-03-13 00:41:09,2020-03-13 00:40:00
2466,2020-03-13 10:52:12+09:00,2020-03-13 01:52:12+00:00,2020-03-13,「投資家はパニック状態…」NY株2352ドル安、ブラックマンデー以来の下落率 - 毎日新聞 ...,「投資家はパニック状態…」NY株2352ドル安、ブラックマンデー以来の下落率 - 毎日新聞 ...,BUSSINESS,2020-03-13 10:52:12,2020-03-13 10:50:00
3083,2020-03-17 00:29:35+09:00,2020-03-16 15:29:35+00:00,2020-03-17,NYダウ2200ドル以上急落 一時取引停止(20/03/16) - ANNnewsCH,NYダウ2200ドル以上急落 一時取引停止(20/03/16) ANNnewsCHダウ先物...,BUSSINESS,2020-03-17 00:29:35,2020-03-17 00:25:00
3091,2020-03-16 23:20:00+09:00,2020-03-16 14:20:00+00:00,2020-03-16,NY株が前週末比2千ドル超急落 取引を一時停止 - livedoor,NY株が前週末比2千ドル超急落 取引を一時停止 livedoorゼロ金利復活も・・・ NY...,BUSSINESS,2020-03-16 23:20:00,2020-03-16 23:20:00
3092,2020-03-16 23:06:24+09:00,2020-03-16 14:06:24+00:00,2020-03-16,NY株暴落、一時2700ドル超安 「ブレーカー」発動 - 朝日新聞,NY株暴落、一時2700ドル超安 「ブレーカー」発動 朝日新聞米株式市場で新型コロナにより...,BUSSINESS,2020-03-16 23:06:24,2020-03-16 23:05:00


In [145]:
df_news[df_news.summary.str.contains('量的緩和')]

,published,published_gmt,published_date,title,summary,topic,published_jst,published_jst_5min
2058,2020-03-13 00:41:18+09:00,2020-03-12 15:41:18+00:00,2020-03-13,ECB総裁、新型コロナ「経済に重大な影響」 緩和限界に - 日本経済新聞,ECB総裁、新型コロナ「経済に重大な影響」 緩和限界に 日本経済新聞ＥＣＢが量的緩和拡大、...,BUSSINESS,2020-03-13 00:41:18,2020-03-13 00:40:00
2071,2020-03-13 04:23:12+09:00,2020-03-12 19:23:12+00:00,2020-03-13,FRBが1.5兆ドル供給 国債購入も、量的緩和に近づく - 日本経済新聞,FRBが1.5兆ドル供給 国債購入も、量的緩和に近づく 日本経済新聞Google ニュース...,BUSSINESS,2020-03-13 04:23:12,2020-03-13 04:20:00
2801,2020-03-16 06:12:50+09:00,2020-03-15 21:12:50+00:00,2020-03-16,FRBが1%緊急利下げ ゼロ金利に、量的緩和も再開 - 日本経済新聞,FRBが1%緊急利下げ ゼロ金利に、量的緩和も再開 日本経済新聞米ＦＲＢ、再び緊急利下げ…...,BUSSINESS,2020-03-16 06:12:50,2020-03-16 06:10:00
3084,2020-03-16 22:31:18+09:00,2020-03-16 13:31:18+00:00,2020-03-16,NYダウ一時2700ドル超安 再び売買停止、世界株安加速 - 日本経済新聞,NYダウ一時2700ドル超安 再び売買停止、世界株安加速 日本経済新聞ＮＹ株暴落、一時２７...,BUSSINESS,2020-03-16 22:31:18,2020-03-16 22:30:00
3091,2020-03-16 23:20:00+09:00,2020-03-16 14:20:00+00:00,2020-03-16,NY株が前週末比2千ドル超急落 取引を一時停止 - livedoor,NY株が前週末比2千ドル超急落 取引を一時停止 livedoorゼロ金利復活も・・・ NY...,BUSSINESS,2020-03-16 23:20:00,2020-03-16 23:20:00
3300,2020-03-17 09:18:00+09:00,2020-03-17 00:18:00+00:00,2020-03-17,Chosun Online - chosunonline.com,Chosun Online chosunonline.com韓経：「韓米通貨スワップ締結…...,BUSSINESS,2020-03-17 09:18:00,2020-03-17 09:15:00
3485,2020-03-17 19:13:03+09:00,2020-03-17 10:13:03+00:00,2020-03-17,「リーマン」よりやっかい、コロナ株安に政策のジレンマ - ロイター (Reuters Japan),「リーマン」よりやっかい、コロナ株安に政策のジレンマ ロイター (Reuters Japa...,BUSSINESS,2020-03-17 19:13:03,2020-03-17 19:10:00
4762,2020-03-23 18:00:00+09:00,2020-03-23 09:00:00+00:00,2020-03-23,FRB、資金供給4兆ドルに 社債購入案も - 日本経済新聞,FRB、資金供給4兆ドルに 社債購入案も 日本経済新聞速報 仮想通貨ビットコイン急騰、米F...,BUSSINESS,2020-03-23 18:00:00,2020-03-23 18:00:00
4763,2020-03-23 21:14:21+09:00,2020-03-23 12:14:21+00:00,2020-03-23,FRB、量的緩和を無制限に 資金供給を大幅拡大 - 日本経済新聞,FRB、量的緩和を無制限に 資金供給を大幅拡大 日本経済新聞ＦＲＢ、経済支援策の第２弾発表...,BUSSINESS,2020-03-23 21:14:21,2020-03-23 21:10:00
4817,2020-03-23 21:35:49+09:00,2020-03-23 12:35:49+00:00,2020-03-23,NYダウ先物反発、一時600ドル高 FRB無制限量的緩和で - 日本経済新聞,NYダウ先物反発、一時600ドル高 FRB無制限量的緩和で 日本経済新聞速報 仮想通貨ビッ...,BUSSINESS,2020-03-23 21:35:49,2020-03-23 21:35:00


In [163]:
def evaluate_sentence_score(
    sentence,
    df_word_score,
    exclude_hinshi_list = ['記号', 'サ変接続', '助詞'],
    debug = False,
):
    return_cols = [cols[0] for cols in df_word_score.columns if 'return' in cols[0]]
    scores = {col: 0 for col in return_cols}
    # words = m_wakati.parse(sentence).split()
    sentence = preprocess_sentence(sentence)
    df_word = pd.DataFrame(
        [item.split('\t') for item in m_chasen.parse(sentence).split('\n') if item not in ('EOS', '')],
        columns=['表層形', '発音', '原型', '形態素の品詞型', '活用形', '活用型']
    )
    df_word = df_word[~df_word['形態素の品詞型'].map(lambda x: any( [hinshi in x for hinshi in exclude_hinshi_list] ))]
    df_word = df_word[df_word['表層形'].str.contains(r"[一-龯ぁ-んァ-ンA-Za-z々ゝゞヽヾヴヵヶ]")]
    for return_col in return_cols:
        word_cnt = 0
        for word in df_word['表層形']:
            try:
                sum = df_word_score.loc[word, (return_col, 'sum')]
                count = df_word_score.loc[word, (return_col, 'count')]
                mean = df_word_score.loc[word, (return_col, 'mean')]
                word_cnt += 1
            except:
                continue
            scores[return_col] += mean
            if debug:
                print(f'{return_col} : {word} , mean:{mean} , count:{count}, sum:{sum}')
        try:
            scores[return_col] /= word_cnt
        except Exception as e:
            print(e)
            scores[return_col] = 0.0
    df_scores = pd.DataFrame({
        'lag_hour': [int(name.split('_')[1].strip('h')) for name in scores.keys()],
        'score': list(scores.values())
    })
    return df_scores

In [153]:
df_sentence_return_scores = evaluate_sentence_score(
    'FRBが1%緊急利下げ ゼロ金利に、量的緩和も再開 日本経済新聞米ＦＲＢ、再び緊急利下げ',
    df_word_score,
    debug=True
)

return_1h : frb , mean:-0.0007057662429376316 , count:54.0, sum:-0.038111377118632106
return_1h : 緊急 , mean:0.0009509865934237101 , count:258.0, sum:0.24535454110331723
return_1h : ゼロ , mean:-0.0013103789991041647 , count:268.0, sum:-0.35118157175991616
return_1h : 金利 , mean:-0.00279411214436491 , count:27.0, sum:-0.07544102789785256
return_1h : 量的緩和 , mean:-0.002667966468717494 , count:12.0, sum:-0.032015597624609926
return_1h : 日本経済 , mean:-0.0005521932119039262 , count:1163.0, sum:-0.6422007054442662
return_1h : 新聞 , mean:-0.0006739005793525538 , count:1338.0, sum:-0.901678975173717
return_1h : 米 , mean:-0.0008593029200399855 , count:1409.0, sum:-1.2107578143363396
return_1h : frb , mean:-0.0007057662429376316 , count:54.0, sum:-0.038111377118632106
return_1h : 緊急 , mean:0.0009509865934237101 , count:258.0, sum:0.24535454110331723
return_1h : frb , mean:-0.0007057662429376316 , count:54.0, sum:-0.038111377118632106
return_1h : 緊急 , mean:0.0009509865934237101 , count:258.0, sum:0.245

In [151]:
df_sentence_return_scores

,lag_hour,score
0,1,-0.000929
1,3,0.004759
2,6,0.007227
3,12,0.009827
4,18,0.011841
5,24,0.013897
6,48,0.019855
7,72,0.039712
8,120,0.046775
9,168,0.067033


In [154]:
df_word_return_train = df_word_return[df_word_return.datetime < '2020-07-01']
df_word_return_test = df_word_return[df_word_return.datetime > '2020-07-01']

In [155]:
df_word_return_train

,word,datetime,size_buy,price_buy,size_sell,price_sell,return_1h,return_3h,return_6h,return_12h,return_18h,return_24h,return_48h,return_72h,return_120h,return_168h,return_720h,return_1440h,return_2160h
0,西海岸,2020-03-02 08:50:00,0.009650,935861.136364,0.013557,936034.857143,0.001318,0.006079,0.016411,0.029423,0.024041,0.015295,0.013133,0.023010,0.027845,-0.126848,-0.272640,0.005865,0.098565
1,探し,2020-03-02 08:50:00,0.009650,935861.136364,0.013557,936034.857143,0.001318,0.006079,0.016411,0.029423,0.024041,0.015295,0.013133,0.023010,0.027845,-0.126848,-0.272640,0.005865,0.098565
2,パカパカスマホ,2020-03-02 08:50:00,0.009650,935861.136364,0.013557,936034.857143,0.001318,0.006079,0.016411,0.029423,0.024041,0.015295,0.013133,0.023010,0.027845,-0.126848,-0.272640,0.005865,0.098565
3,台風,2020-03-02 08:50:00,0.009650,935861.136364,0.013557,936034.857143,0.001318,0.006079,0.016411,0.029423,0.024041,0.015295,0.013133,0.023010,0.027845,-0.126848,-0.272640,0.005865,0.098565
4,掘り出し物,2020-03-02 08:50:00,0.009650,935861.136364,0.013557,936034.857143,0.001318,0.006079,0.016411,0.029423,0.024041,0.015295,0.013133,0.023010,0.027845,-0.126848,-0.272640,0.005865,0.098565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682110,見る,2020-06-30 16:35:00,0.003733,989581.666667,0.005800,989100.000000,-0.003098,-0.002217,-0.002558,-0.003771,-0.004773,0.008571,-0.018898,-0.010686,-0.015651,0.007309,0.167274,0.227342,0.161602
682111,対象,2020-06-30 16:35:00,0.003733,989581.666667,0.005800,989100.000000,-0.003098,-0.002217,-0.002558,-0.003771,-0.004773,0.008571,-0.018898,-0.010686,-0.015651,0.007309,0.167274,0.227342,0.161602
682112,トレンド,2020-06-30 16:35:00,0.003733,989581.666667,0.005800,989100.000000,-0.003098,-0.002217,-0.002558,-0.003771,-0.004773,0.008571,-0.018898,-0.010686,-0.015651,0.007309,0.167274,0.227342,0.161602
682113,中華,2020-06-30 16:35:00,0.003733,989581.666667,0.005800,989100.000000,-0.003098,-0.002217,-0.002558,-0.003771,-0.004773,0.008571,-0.018898,-0.010686,-0.015651,0.007309,0.167274,0.227342,0.161602


In [156]:
df_word_return_test

,word,datetime,size_buy,price_buy,size_sell,price_sell,return_1h,return_3h,return_6h,return_12h,return_18h,return_24h,return_48h,return_72h,return_120h,return_168h,return_720h,return_1440h,return_2160h
676528,リニア,2020-07-01 06:00:00,0.095661,9.856014e+05,0.097405,9.850867e+05,0.000096,0.001105,0.00062,0.011768,0.006744,0.005026,-0.005332,-0.007669,-0.003400,0.012336,0.170900,0.242012,0.143667
676529,現代ビジネス,2020-07-01 06:00:00,0.095661,9.856014e+05,0.097405,9.850867e+05,0.000096,0.001105,0.00062,0.011768,0.006744,0.005026,-0.005332,-0.007669,-0.003400,0.012336,0.170900,0.242012,0.143667
676530,吹き飛ばす,2020-07-01 06:00:00,0.095661,9.856014e+05,0.097405,9.850867e+05,0.000096,0.001105,0.00062,0.011768,0.006744,0.005026,-0.005332,-0.007669,-0.003400,0.012336,0.170900,0.242012,0.143667
676531,天才,2020-07-01 06:00:00,0.095661,9.856014e+05,0.097405,9.850867e+05,0.000096,0.001105,0.00062,0.011768,0.006744,0.005026,-0.005332,-0.007669,-0.003400,0.012336,0.170900,0.242012,0.143667
676532,柔道家,2020-07-01 06:00:00,0.095661,9.856014e+05,0.097405,9.850867e+05,0.000096,0.001105,0.00062,0.011768,0.006744,0.005026,-0.005332,-0.007669,-0.003400,0.012336,0.170900,0.242012,0.143667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999960,コロナ感染,2020-08-22 22:40:00,0.049200,1.235480e+06,0.077467,1.234866e+06,0.001348,-0.001484,-0.00419,-0.007801,-0.003209,-0.000112,0.008191,-0.019560,-0.026116,-0.017224,-0.107938,0.106727,0.563611
999961,諦め,2020-08-22 22:40:00,0.049200,1.235480e+06,0.077467,1.234866e+06,0.001348,-0.001484,-0.00419,-0.007801,-0.003209,-0.000112,0.008191,-0.019560,-0.026116,-0.017224,-0.107938,0.106727,0.563611
999962,見る,2020-08-22 22:40:00,0.049200,1.235480e+06,0.077467,1.234866e+06,0.001348,-0.001484,-0.00419,-0.007801,-0.003209,-0.000112,0.008191,-0.019560,-0.026116,-0.017224,-0.107938,0.106727,0.563611
999963,.jp,2020-08-22 22:40:00,0.049200,1.235480e+06,0.077467,1.234866e+06,0.001348,-0.001484,-0.00419,-0.007801,-0.003209,-0.000112,0.008191,-0.019560,-0.026116,-0.017224,-0.107938,0.106727,0.563611


In [157]:
df_word_score_train = create_word_score_df(df_word_return_train)

In [158]:
df_news_test = df_news[df_news['published_date'] > '2020-07-01']

In [159]:
df_news_test

,published,published_gmt,published_date,title,summary,topic,published_jst,published_jst_5min
28712,2020-07-02 06:00:00+09:00,2020-07-01 21:00:00+00:00,2020-07-02,167センチ山本で中日今季初連勝 6イニング２失点 令和の虎キラーが竜に勢い - 中日スポー...,167センチ山本で中日今季初連勝 6イニング２失点 令和の虎キラーが竜に勢い 中日スポーツ...,SPORTS,2020-07-02 06:00:00,2020-07-02 06:00:00
28717,2020-07-02 00:18:41+09:00,2020-07-01 15:18:41+00:00,2020-07-02,2020年夏アニメ作品リスト - ナタリー,2020年夏アニメ作品リスト ナタリー2020年夏アニメは様々な危機に⽴ち向かう“⼥の⼦”...,ENTERTAINMENT,2020-07-02 00:18:41,2020-07-02 00:15:00
28718,2020-07-02 08:00:00+09:00,2020-07-01 23:00:00+00:00,2020-07-02,20年ぶり復活のロボコン、中華屋の天井を破って登場する本予告編解禁 - AV Watch,20年ぶり復活のロボコン、中華屋の天井を破って登場する本予告編解禁 AV Watchがんば...,ENTERTAINMENT,2020-07-02 08:00:00,2020-07-02 08:00:00
28726,2020-07-02 08:29:00+09:00,2020-07-01 23:29:00+00:00,2020-07-02,5G対応の次世代「iPhone」量産に2カ月程度の遅れ--新型コロナの影響 - CNET J...,5G対応の次世代「iPhone」量産に2カ月程度の遅れ--新型コロナの影響 CNET Ja...,TECHNOLOGY,2020-07-02 08:29:00,2020-07-02 08:25:00
28728,2020-07-02 05:55:24+09:00,2020-07-01 20:55:24+00:00,2020-07-02,60秒の長編「ゼスプリ」CMが断トツ人気のワケ（東洋経済オンライン） - Yahoo!ニュース,60秒の長編「ゼスプリ」CMが断トツ人気のワケ（東洋経済オンライン） Yahoo!ニュース...,BUSSINESS,2020-07-02 05:55:24,2020-07-02 05:55:00
...,...,...,...,...,...,...,...,...
42119,2020-08-22 10:33:22+09:00,2020-08-22 01:33:22+00:00,2020-08-22,コロナ終息は「2年以内」 WHOテドロス氏が言及 [新型コロナウイルス] - 朝日新聞デジタル,コロナ終息は「2年以内」 WHOテドロス氏が言及 [新型コロナウイルス] 朝日新聞デジタル...,WORLD,2020-08-22 10:33:22,2020-08-22 10:30:00
42120,2020-08-22 10:33:22+09:00,2020-08-22 01:33:22+00:00,2020-08-22,コロナ終息は「2年未満で」 WHOテドロス氏が言及 [新型コロナウイルス] - 朝日新聞デジタル,コロナ終息は「2年未満で」 WHOテドロス氏が言及 [新型コロナウイルス] 朝日新聞デジタ...,WORLD,2020-08-22 10:33:22,2020-08-22 10:30:00
42121,2020-08-22 12:10:00+09:00,2020-08-22 03:10:00+00:00,2020-08-22,サツキは児童虐待？「となりのトトロ」2年ぶりの地上波放送で、疑問の声噴出 - リアルライブ,サツキは児童虐待？「となりのトトロ」2年ぶりの地上波放送で、疑問の声噴出 リアルライブ,ENTERTAINMENT,2020-08-22 12:10:00,2020-08-22 12:10:00
42122,2020-08-22 12:10:00+09:00,2020-08-22 03:10:00+00:00,2020-08-22,サツキは児童虐待？「となりのトトロ」放送で時流を反映した指摘 - livedoor,サツキは児童虐待？「となりのトトロ」放送で時流を反映した指摘 livedoor「となりのト...,ENTERTAINMENT,2020-08-22 12:10:00,2020-08-22 12:10:00


In [164]:
scores = {}

for summary in pb(df_news_test['summary']):
    df_score = evaluate_sentence_score(summary, df_word_score_train)
    for hour, score in zip(df_score['lag_hour'], df_score['score']):
        if hour not in scores:
            scores[hour] = []
        scores[hour].append(score)

division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero
division by zero
float division by zero
float division by zero


In [165]:
for key, score in scores.items():
    df_news_test[f'score_{key}'] = score

In [166]:
df_news_test

,published,published_gmt,published_date,title,summary,topic,published_jst,published_jst_5min,score_1,score_3,score_6,score_12,score_18,score_24,score_48,score_72,score_120,score_168,score_720,score_1440,score_2160
28712,2020-07-02 06:00:00+09:00,2020-07-01 21:00:00+00:00,2020-07-02,167センチ山本で中日今季初連勝 6イニング２失点 令和の虎キラーが竜に勢い - 中日スポー...,167センチ山本で中日今季初連勝 6イニング２失点 令和の虎キラーが竜に勢い 中日スポーツ...,SPORTS,2020-07-02 06:00:00,2020-07-02 06:00:00,-0.000875,-0.001573,-0.001443,-0.001328,-0.001946,-0.004834,0.001217,0.002352,0.009406,0.004667,0.093615,0.269551,0.273662
28717,2020-07-02 00:18:41+09:00,2020-07-01 15:18:41+00:00,2020-07-02,2020年夏アニメ作品リスト - ナタリー,2020年夏アニメ作品リスト ナタリー2020年夏アニメは様々な危機に⽴ち向かう“⼥の⼦”...,ENTERTAINMENT,2020-07-02 00:18:41,2020-07-02 00:15:00,-0.000834,-0.000583,-0.000769,-0.000589,-0.000472,-0.001129,0.000121,0.003236,0.006374,0.013843,0.116805,0.253449,0.292336
28718,2020-07-02 08:00:00+09:00,2020-07-01 23:00:00+00:00,2020-07-02,20年ぶり復活のロボコン、中華屋の天井を破って登場する本予告編解禁 - AV Watch,20年ぶり復活のロボコン、中華屋の天井を破って登場する本予告編解禁 AV Watchがんば...,ENTERTAINMENT,2020-07-02 08:00:00,2020-07-02 08:00:00,0.000600,0.001050,0.000586,0.000233,0.001498,0.000339,0.003618,0.003439,0.008991,0.012280,0.094634,0.233958,0.280466
28726,2020-07-02 08:29:00+09:00,2020-07-01 23:29:00+00:00,2020-07-02,5G対応の次世代「iPhone」量産に2カ月程度の遅れ--新型コロナの影響 - CNET J...,5G対応の次世代「iPhone」量産に2カ月程度の遅れ--新型コロナの影響 CNET Ja...,TECHNOLOGY,2020-07-02 08:29:00,2020-07-02 08:25:00,-0.000798,-0.000527,-0.000721,0.000352,-0.000151,-0.000967,-0.000653,0.000310,0.002935,0.013558,0.120020,0.266562,0.308840
28728,2020-07-02 05:55:24+09:00,2020-07-01 20:55:24+00:00,2020-07-02,60秒の長編「ゼスプリ」CMが断トツ人気のワケ（東洋経済オンライン） - Yahoo!ニュース,60秒の長編「ゼスプリ」CMが断トツ人気のワケ（東洋経済オンライン） Yahoo!ニュース...,BUSSINESS,2020-07-02 05:55:24,2020-07-02 05:55:00,-0.001416,0.000679,0.000749,0.003806,0.002973,0.002885,0.007321,0.008404,0.010069,0.018445,0.140926,0.266482,0.294038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42119,2020-08-22 10:33:22+09:00,2020-08-22 01:33:22+00:00,2020-08-22,コロナ終息は「2年以内」 WHOテドロス氏が言及 [新型コロナウイルス] - 朝日新聞デジタル,コロナ終息は「2年以内」 WHOテドロス氏が言及 [新型コロナウイルス] 朝日新聞デジタル...,WORLD,2020-08-22 10:33:22,2020-08-22 10:30:00,-0.000679,-0.000225,-0.001652,-0.000548,-0.001882,-0.000624,0.001263,0.004166,0.012002,0.019368,0.121730,0.251112,0.297469
42120,2020-08-22 10:33:22+09:00,2020-08-22 01:33:22+00:00,2020-08-22,コロナ終息は「2年未満で」 WHOテドロス氏が言及 [新型コロナウイルス] - 朝日新聞デジタル,コロナ終息は「2年未満で」 WHOテドロス氏が言及 [新型コロナウイルス] 朝日新聞デジタ...,WORLD,2020-08-22 10:33:22,2020-08-22 10:30:00,-0.000690,-0.000364,-0.001426,-0.000307,-0.001612,-0.000931,0.001864,0.004862,0.012163,0.021460,0.127315,0.245654,0.293346
42121,2020-08-22 12:10:00+09:00,2020-08-22 03:10:00+00:00,2020-08-22,サツキは児童虐待？「となりのトトロ」2年ぶりの地上波放送で、疑問の声噴出 - リアルライブ,サツキは児童虐待？「となりのトトロ」2年ぶりの地上波放送で、疑問の声噴出 リアルライブ,ENTERTAINMENT,2020-08-22 12:10:00,2020-08-22 12:10:00,0.003050,-0.001238,-0.002233,0.005076,0.000722,-0.003033,0.000015,-0.003394,0.041285,0.046941,0.146268,0.395241,0.392231
42122,2020-08-22 12:10:00+09:00,2020-08-22 03:10:00+00:00,2020-08-22,サツキは児童虐待？「となりのトトロ」放送で時流を反映した指摘 - livedoor,サツキは児童虐待？「となりのトトロ」放送で時流を反映した指摘 livedoor「となりのト...,ENTERTAINMENT,2020-08-22 12:10:00,2020-08-22 12:10:00,0.000817,0.000179,-0.000669,0.003428,0.000569,-0.000742,0.002740,0.007044,0.026604,0.021987,0.115661,0.271993,0.302670


In [167]:
df_news_test.set_index('published_jst', inplace=True)

In [169]:
df_pred_score = pd.merge(
    df_news_test.resample('5T').mean(),
    df_btc_return,
    how='inner',
    left_index=True, right_index=True
)

In [170]:
df_pred_score

,score_1,score_3,score_6,score_12,score_18,score_24,score_48,score_72,score_120,score_168,score_720,score_1440,score_2160,size_buy,price_buy,size_sell,price_sell,return_1h,return_3h,return_6h,return_12h,return_18h,return_24h,return_48h,return_72h,return_120h,return_168h,return_720h,return_1440h,return_2160h
2020-07-02 00:00:00,-0.000099,0.000301,0.000903,0.000903,0.001156,0.003162,0.005243,0.007839,0.014668,0.026704,0.126003,0.265594,0.302543,0.010000,9.931967e+05,0.033750,9.922488e+05,-0.000408,0.002158,-0.002660,-0.004778,-0.017364,-0.015908,-0.018930,-0.010585,0.010147,0.018963,0.213850,0.244702,0.153351
2020-07-02 00:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005200,9.935950e+05,0.024167,9.923845e+05,-0.000628,0.001699,-0.004061,-0.005178,-0.018965,-0.017256,-0.018413,-0.010658,0.009157,0.018547,0.211968,0.243844,0.152855
2020-07-02 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003100,9.935488e+05,0.014583,9.925202e+05,-0.000401,0.001689,-0.004289,-0.004977,-0.018276,-0.017059,-0.018709,-0.010617,0.011456,0.017449,0.209476,0.243831,0.152901
2020-07-02 00:15:00,-0.000834,-0.000583,-0.000769,-0.000589,-0.000472,-0.001129,0.000121,0.003236,0.006374,0.013843,0.116805,0.253449,0.292336,0.001000,9.935025e+05,0.005000,9.926560e+05,-0.000174,0.001678,-0.004517,-0.004776,-0.017586,-0.016862,-0.018457,-0.010545,0.011709,0.016610,0.208263,0.242316,0.152243
2020-07-02 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011500,9.940412e+05,0.100000,9.928860e+05,-0.000536,0.001034,-0.004744,-0.005160,-0.017475,-0.016329,-0.018783,-0.011054,0.011212,0.016123,0.206703,0.241321,0.150908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-23 02:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005750,1.231112e+06,0.034800,1.231111e+06,-0.000895,-0.003614,-0.002433,-0.001227,0.004925,0.002510,0.006754,-0.013390,-0.012273,-0.004147,-0.112930,0.087851,0.589147
2020-08-23 02:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012500,1.231014e+06,0.329000,1.230450e+06,-0.000792,-0.003273,-0.003089,-0.000977,0.004766,0.002179,0.005566,-0.013104,-0.012683,-0.004104,-0.111581,0.088326,0.586407
2020-08-23 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.130830,1.230164e+06,0.077075,1.229880e+06,-0.000078,-0.002322,-0.003135,0.000339,0.004729,0.003228,0.005507,-0.014129,-0.011438,-0.003453,-0.109688,0.089466,0.582680
2020-08-23 03:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.068882,1.230904e+06,0.093874,1.230235e+06,-0.000655,-0.002660,-0.001291,0.000305,0.004027,0.002982,0.006864,-0.015009,-0.011393,-0.004462,-0.110332,0.088188,0.585003


In [171]:
df_pred_score.dropna()

,score_1,score_3,score_6,score_12,score_18,score_24,score_48,score_72,score_120,score_168,score_720,score_1440,score_2160,size_buy,price_buy,size_sell,price_sell,return_1h,return_3h,return_6h,return_12h,return_18h,return_24h,return_48h,return_72h,return_120h,return_168h,return_720h,return_1440h,return_2160h
2020-07-02 00:00:00,-0.000099,0.000301,0.000903,0.000903,0.001156,0.003162,0.005243,0.007839,0.014668,0.026704,0.126003,0.265594,0.302543,0.010000,9.931967e+05,0.033750,9.922488e+05,-0.000408,0.002158,-0.002660,-0.004778,-0.017364,-0.015908,-0.018930,-0.010585,0.010147,0.018963,0.213850,0.244702,0.153351
2020-07-02 00:15:00,-0.000834,-0.000583,-0.000769,-0.000589,-0.000472,-0.001129,0.000121,0.003236,0.006374,0.013843,0.116805,0.253449,0.292336,0.001000,9.935025e+05,0.005000,9.926560e+05,-0.000174,0.001678,-0.004517,-0.004776,-0.017586,-0.016862,-0.018457,-0.010545,0.011709,0.016610,0.208263,0.242316,0.152243
2020-07-02 00:25:00,0.000446,0.001737,0.001988,0.001306,0.004466,0.006597,-0.002988,-0.001008,0.000944,0.015078,0.107491,0.255261,0.273291,0.022000,9.945800e+05,0.195000,9.931160e+05,-0.000897,0.000390,-0.004872,-0.005545,-0.017793,-0.016434,-0.019186,-0.012972,0.010715,0.014768,0.201446,0.240821,0.150246
2020-07-02 00:30:00,0.000798,0.002404,0.002297,0.000794,0.003426,0.005811,-0.004083,-0.001059,-0.000348,0.012968,0.105428,0.254018,0.269612,0.009650,9.925500e+05,0.018200,9.926360e+05,0.001327,0.002334,-0.002425,0.000417,-0.015666,-0.013994,-0.017051,-0.009982,0.011452,0.016489,0.203394,0.244008,0.152637
2020-07-02 00:55:00,-0.000499,-0.001706,-0.002749,-0.001164,-0.003933,-0.002096,0.000091,0.000117,0.004909,0.011071,0.123315,0.281189,0.317396,0.105000,9.925125e+05,0.033583,9.926127e+05,0.002268,0.001499,-0.001776,0.002116,-0.016834,-0.012787,-0.016749,-0.010088,0.009573,0.017622,0.203824,0.243803,0.152932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-23 00:00:00,0.000661,0.006429,0.006774,0.007778,0.001295,0.002823,0.005965,-0.004575,0.010165,0.026738,0.138785,0.325982,0.333047,0.310350,1.237232e+06,0.039900,1.236975e+06,-0.003069,-0.005943,-0.008022,-0.011757,-0.002425,-0.002467,0.008138,-0.024958,-0.023920,-0.021392,-0.120107,0.086853,0.567617
2020-08-23 00:45:00,-0.000046,0.000631,-0.000976,-0.000484,-0.003541,-0.005355,-0.003156,-0.006576,-0.001979,0.002819,0.074683,0.218080,0.239559,0.015150,1.234149e+06,0.050000,1.233750e+06,-0.000211,-0.003614,-0.004544,-0.007089,0.000399,-0.002757,0.009653,-0.025974,-0.020297,-0.015773,-0.111924,0.092777,0.581139
2020-08-23 02:05:00,0.000783,0.000797,0.000195,0.001142,0.001952,0.002047,0.002473,0.007063,0.004710,0.006464,0.100044,0.235725,0.268412,0.008000,1.234100e+06,0.011933,1.233551e+06,-0.003131,-0.004878,-0.003322,-0.003655,0.003392,-0.000294,0.008286,-0.018155,-0.014933,-0.007566,-0.115211,0.089577,0.590219
2020-08-23 02:35:00,-0.000699,-0.001423,-0.000815,-0.000491,-0.006257,-0.005677,-0.005349,-0.002940,-0.001057,0.003508,0.073573,0.232618,0.267542,0.005000,1.231898e+06,0.081233,1.231032e+06,-0.003062,-0.003651,-0.003275,-0.002446,0.005001,0.002683,0.007853,-0.016152,-0.013310,-0.005413,-0.112597,0.083129,0.591123


In [172]:
(df_pred_score.dropna()['score_24'] * df_pred_score.dropna()['return_24h'] > 0).sum() / len(df_pred_score.dropna())

0.52875

In [173]:
(df_pred_score.dropna()['score_12'] * df_pred_score.dropna()['return_12h'] > 0).sum() / len(df_pred_score.dropna())

0.4926388888888889

In [174]:
df_score['lag_hour']

0        1
1        3
2        6
3       12
4       18
5       24
6       48
7       72
8      120
9      168
10     720
11    1440
12    2160
Name: lag_hour, dtype: int64

In [176]:
for lag_hour in df_score['lag_hour']:
    acc = (df_pred_score.dropna()[f'score_{lag_hour}'] * df_pred_score.dropna()[f'return_{lag_hour}h'] > 0).sum() / len(df_pred_score.dropna())
    print(f'{lag_hour}h lag : {acc}')

1h lag : 0.5186111111111111
3h lag : 0.50125
6h lag : 0.50375
12h lag : 0.4926388888888889
18h lag : 0.51
24h lag : 0.52875
48h lag : 0.5459722222222222
72h lag : 0.5613888888888889
120h lag : 0.59875
168h lag : 0.6265277777777778
720h lag : 0.6316666666666667
1440h lag : 0.5318055555555555
2160h lag : 0.9998611111111111
